In [9]:
!pip install pyspark

In [10]:
from google.colab import files
uploaded = files.upload()

Saving insurance_data.csv to insurance_data (1).csv


In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count

# Create Spark session
spark = SparkSession.builder.master("local[*]").appName("InsuranceDataAnalysis").getOrCreate()

# Read CSV
df = spark.read.option("header", True).option("inferSchema", True).csv(r"insurance_data.csv")

# Show schema & first few rows
df.printSchema()
df.show(5)

# 1. Average claim by region
df.groupBy("region").agg(avg(col("claim")).alias("Avg_Claim")) \
    .orderBy(col("Avg_Claim").desc()) \
    .show()

# 2. Count of smokers vs non-smokers
df.groupBy("smoker").count().show()

# 3. Average BMI by diabetic status
df.groupBy("diabetic").agg(avg(col("bmi")).alias("Avg_BMI")).show()

# 4. Average claim by gender
df.groupBy("gender").agg(avg(col("claim")).alias("Avg_Claim")).show()

# Save as Parquet
df.write.mode("overwrite").parquet("output_parquet")

print("Analysis complete.")

spark.stop()


root
 |-- index: integer (nullable = true)
 |-- PatientID: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- bloodpressure: integer (nullable = true)
 |-- diabetic: string (nullable = true)
 |-- children: integer (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- claim: double (nullable = true)

+-----+---------+----+------+----+-------------+--------+--------+------+---------+-------+
|index|PatientID| age|gender| bmi|bloodpressure|diabetic|children|smoker|   region|  claim|
+-----+---------+----+------+----+-------------+--------+--------+------+---------+-------+
|    0|        1|  39|  male|23.2|           91|     Yes|       0|    No|southeast|1121.87|
|    1|        2|  24|  male|30.1|           87|      No|       0|    No|southeast|1131.51|
|    2|        3|NULL|  male|33.3|           82|     Yes|       0|    No|southeast|1135.94|
|    3|      